# CatBoost + ClearML Demo: Titanic Survival Prediction

Этот проект демонстрирует интеграцию CatBoost с ClearML для отслеживания экспериментов машинного обучения.

Что делает этот проект:
- Загружает датасет Titanic
- Выполняет базовую предобработку данных
- Обучает модель CatBoost для предсказания выживания пассажиров
- Использует ClearML для логирования экспериментов, метрик и артефактов
- Выполняет гиперпараметрическую оптимизацию

Технологии:
- CatBoost: градиентный бустинг для категориальных признаков
- ClearML: платформа для отслеживания ML экспериментов
- Pandas/NumPy: обработка данных
- Scikit-learn: разделение данных и гиперпараметрическая оптимизация
"""

Настройка окружения ClearML (замените на свои ключи доступа к ClearML серверу)

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY='Ваш ключ доступа'
%env CLEARML_API_SECRET_KEY='Ваш секретный ключ'

Импорт требуемых библиотек

In [ ]:
from clearml import Task, Logger
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterSampler
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

Инициализация ClearML задачи

In [ ]:
task = Task.init(project_name='OTUS_test',
           task_name='CatBoost_notebook_v1',
           tags=['Model:CatBoost', 'type:notebook'])

Загрузка и первичный анализ данных, а также загрузка данных как артефакт в ClearML

In [ ]:
df_raw = pd.read_csv('titanic.csv')

task.upload_artifact(name='data.raw', artifact_object=df_raw)

df_raw.head()

Статистическое описание данных

In [ ]:
task.upload_artifact(name='eda describe', artifact_object=df_raw.describe())
df_raw.describe()

Анализ целевой переменной (Survived)

In [ ]:
task.upload_artifact(name='target_info', artifact_object=df_raw['Survived'].value_counts(normalize=True).reset_index())

Визуализация распределения

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df_raw, x='Survived')
plt.title('Распределение выживших пассажиров')
plt.xlabel('Выжил (1) / Не выжил (0)')
plt.ylabel('Количество')
plt.show()

Предобработка данных

In [ ]:
df_preproc = df_raw.drop(columns=['PassengerId', 'Name', 'Ticket'])
for col in ['Sex', 'Cabin', 'Embarked']:
    df_preproc[col] = df_preproc[col].astype(str)
    
task.upload_artifact(name='data.preproc', artifact_object=df_preproc)


In [ ]:
df_raw.columns

Разделение данных на тренировочную и тестовую выборки, подготовка признаков для модели

In [ ]:
train, test = train_test_split(df_preproc, test_size=0.33, random_state=42)

X_train = train.drop(columns=['Survived']) 
X_train = X_train[['Sex', 'Cabin', 'Embarked']]
y_train = train['Survived']

X_test = test.drop(columns=['Survived']) 
X_test = X_test[['Sex', 'Cabin', 'Embarked']]
y_test = test['Survived']

Настройка параметрической сетки

In [ ]:
param_grid = {
    'depth' : [4, 5, 6, 7, 8, 10, 16],
    'learning_rate': [0.1, 0.05, 0.01, 0.0005, 0.0001],
    'iterations': [30, 50, 100, 150, 200]
}

# На этом этапе также происходит инициализация 
# логгера для записи метрик
log = Logger.current_logger()

best_score = 0 
best_model = None
i = 0

Обучение модели и поиск лучших гиперпараметров

In [ ]:
for param in ParameterSampler(param_grid, n_iter=4, random_state=42):
    print(param)
    parametrs_dict = Task.current_task().connect(param)
    print(i+1)
    
    model = CatBoostClassifier(**param, silent=True)
    model.fit(X_train, y_train, cat_features=['Sex', 'Cabin', 'Embarked'])
    
    #Оценка модели и обучение 
    train_scores = model.eval_metrics(
    data=Pool(X_train, y_train, cat_features=['Sex', 'Cabin', 'Embarked']),
    metrics=['Logloss', 'AUC'])
    train_logloss = round(train_scores['Logloss'][-1], 4)
    train_roc_auc = round(train_scores['AUC'][-1]*100, 4)
    
    test_scores = model.eval_metrics(
    data=Pool(X_test, y_test, cat_features=['Sex', 'Cabin', 'Embarked']),
    metrics=['Logloss', 'AUC'])
    test_logloss = round(test_scores['Logloss'][-1], 4)
    test_roc_auc = round(test_scores['AUC'][-1]*100, 4)
    
    # Сохраняем лучшую модель
    if test_roc_auc > best_score:
        best_score = test_roc_auc
        best_model = model
        
        # Логируем метрики в ClearML
        log.report_scalar('Logloss', 'Test', iteration=i, value=test_logloss)
        log.report_scalar('Logloss', 'Train', iteration=i, value=train_logloss)
        
        log.report_scalar('ROC AUC', 'Test', iteration=i, value=test_roc_auc)
        log.report_scalar('ROC AUC', 'Train', iteration=i, value=train_roc_auc)
        
        i+=1

Логируем финальные результаты в ClearML

In [ ]:
log.report_single_value(name='Best ROC AUC', value=test_roc_auc)
log.report_single_value(name='Best Logloss', value=test_logloss)
log.report_single_value(name='Train Rows', value=X_train.shape[0])
log.report_single_value(name='Test Rows', value=X_test.shape[0])
log.report_single_value(name='Columns', value=X_train.shape[1])

Сохранение лучшей модели

In [ ]:
best_model_name = 'your_best_model.cbn'
best_model.save_model(best_model_name)
task.upload_artifact(name=best_model_name, artifact_object=best_model)

Завершение эксперимента в ClearMl

In [ ]:
task.close()